In [141]:
!pip install ipyannotations
!pip install pandas
!pip install python-Levenshtein
!pip install beautifulsoup4
!pip install sentencepiece
!pip install tqdm

In [142]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [143]:
en_sk_pipeline = pipeline(task="text2text-generation", model="Helsinki-NLP/opus-mt-en-sk")

In [ ]:
split_name = "val" #one of train, test, val

In [144]:
theTranslatedDataSetDF = pd.read_parquet(f"./translated/conll2003-sk/{split_name}.parquet")
theTranslatedDataSetDF.iloc[3:,:]

,id,tokens,ner_tags,translated
3,3,"[Their, stay, on, top, ,, though, ,, may, be, short-lived, as, title, rivals, Essex, ,, Derbyshire, and, Surrey, all, closed, in, on, victory, while, Kent, made, up, for, lost, time, in, their, rain-affected, match, against, Nottinghamshire, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0]","Ich pobyt na vrchole však môže trvať len krátko, pretože súperi o titul Essex, Derbyshire a Surrey dosiahli víťazstvo, zatiaľ čo Kent dohnal stratený čas v zápase s Nottinghamshire postihnutým dažďom."
4,4,"[After, bowling, Somerset, out, for, 83, on, the, opening, morning, at, Grace, Road, ,, Leicestershire, extended, their, first, innings, by, 94, runs, before, being, bowled, out, for, 296, with, England, discard, Andy, Caddick, taking, three, for, 83, .]","[0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0]","Po bowlingu Somerset za 83 bodov v úvodné ráno na Grace Road, Leicestershire predĺžil svoje prvé smeny o 94 behov, než bol odhodený za 296, pričom Anglicko vyradilo Andyho Caddicka a tri za 83."
5,5,"[Trailing, by, 213, ,, Somerset, got, a, solid, start, to, their, second, innings, before, Simmons, stepped, in, to, bundle, them, out, for, 174, .]","[0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Po 213 , Somerset dostal solídny štart do svojej druhej smeny predtým , než Simmons vstúpil do balíka ich za 174 ."
6,6,"[Essex, ,, however, ,, look, certain, to, regain, their, top, spot, after, Nasser, Hussain, and, Peter, Such, gave, them, a, firm, grip, on, their, match, against, Yorkshire, at, Headingley, .]","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 5, 0]","Zdá sa však, že Essex si je istý, že opäť získa svoje prvé miesto po tom, čo im Nasser Hussain a Peter Such dali pevnú kontrolu nad zápasom proti Yorkshire v Headingley."
7,7,"[Hussain, ,, considered, surplus, to, England, 's, one-day, requirements, ,, struck, 158, ,, his, first, championship, century, of, the, season, ,, as, Essex, reached, 372, and, took, a, first, innings, lead, of, 82, .]","[1, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Hussain, považovaný za prebytok vzhľadom na jednodňové požiadavky Anglicka, dosiahol 158, svoje prvé majstrovské storočie sezóny, keď Essex dosiahol 372 a ujal sa vedenia v prvej smene 82."
...,...,...,...,...
3245,3245,"[But, the, prices, may, move, in, a, close, range, following, a, continued, market, uptrend, ., ""]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Ceny sa však môžu pohybovať v blízkom rozmedzí po pokračujúcom vzostupnom trende trhu. &quot;
3246,3246,"[Brokers, said, blue, chips, like, IDLC, ,, Bangladesh, Lamps, ,, Chittagong, Cement, and, Atlas, Bangladesh, were, expected, to, rise, .]","[0, 0, 0, 0, 0, 3, 0, 3, 4, 0, 3, 4, 0, 3, 4, 0, 0, 0, 0, 0]","Makléri uviedli, že blue chips ako IDLC, Bangladesh Lamps, Chittagong Cement a Atlas Bangladesh sa očakáva nárast."
3247,3247,"[They, said, there, was, still, demand, for, blue, chips, in, engineering, sector, despite, their, persistent, rise, over, the, past, several, sessions, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Povedali, že v strojárskom sektore stále existuje dopyt po modrých čipsoch, a to aj napriek ich trvalému nárastu počas niekoľkých posledných zasadnutí."
3248,3248,"[The, DSE, all, share, price, index, closed, 2.73, points, or, 0.22, percent, up, at, 1,196.35, on, a, turnover, of, 133.7, million, taka, on, Thursday, .]","[0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Index všetkých cien akcií DSE uzavrel vo štvrtok o 2,73 bodu alebo 0,22 percenta na úrovni 1 196,35 pri obrate 133,7 milióna taka."


In [145]:
from bs4 import BeautifulSoup
ner_tags = ["B-PER (1)", "I-PER (2)", "B-ORG (3)", "I-ORG (4)", "B-LOC (5)", "I-LOC (6)", "B-MISC (7)", "I-MISC (8)", "TODO-FIX"]

def tagId_to_label(tag_id):
    if (tag_id-1)>=0:
        return ner_tags[tag_id-1] #-1 because ner_tags ommits 0-th no-ent tag
    else:
        return None

def label_to_tagId(label):
    if label:
        return ner_tags.index(label)+1  #+1 because ner_tags ommits 0-th no-ent tag
    else:
        return 0


In [146]:

def get_NER_phrases(tokens, ner_tag_ids):
    open_phrase=None
    open_TAGs=None
    #result = []
    for token, tag in zip(tokens,ner_tag_ids):
        if tag>0:
            if tag>0:
                if tag%2==1: #if is odd number, means it starts with B...beginning    
                    if open_phrase: #return previous phrase
                        yield (open_phrase,open_TAGs)
                    open_phrase=token
                    open_TAGs=[tag]
                else:
                    if open_phrase is None:
                        raise Exception(f"Inconsistent annotation labels: {tokens},{ner_tag_ids}, 'I-' label withou previously defined 'B-' label")
                    open_phrase+=(" " +token)
                    open_TAGs.append(tag)
        elif open_phrase: #return previous phrase
            yield (open_phrase,open_TAGs)
            open_phrase=None
            open_TAGs=None

    if open_phrase: #return last phrase
        yield (open_phrase,open_TAGs)
    #return result


#test        
    
list(get_NER_phrases(["this","is","New","York", ",","Peter","New","York"], [0,0,3,4,0,1,3,4]))

[('New York', [3, 4]), ('Peter', [1]), ('New York', [3, 4])]

In [147]:
import Levenshtein
import re

def tokenize(text):
    return ["".join(m) for m in re.findall("(\w+)|(\S)", text)]

def str_to_ent(whole_str, str_part, tags, match_tolerance=0):
    found =whole_str.lower().find(str_part.lower())
    result = []
    if found>=0:
        tokenized_part = tokenize(str_part)
        while found>=0:
            cursor_pos=found
            end=len(whole_str)
            for token,tag in zip(tokenized_part,tags) :
                start = cursor_pos 
                end = start+len(token)
                cursor_pos=end+1 # +1 because " " separator
                result.append( (start,end,tag))
            found =whole_str.lower().find(str_part.lower(),end+1)

    elif match_tolerance>0:
        tokenized = tokenize(whole_str.lower())
        span_len = len(tags)
        best_dist=match_tolerance+999
        best_match =None
        for i in range(len(tokenized)):
            if i+span_len>len(tokenized): #slide while width of phrase allows it ... 
                break
            match_candidate = " ".join(tokenized[i:i+span_len])
            dist = Levenshtein.distance(match_candidate, str_part.lower())
            if dist<=match_tolerance and dist<best_dist:
                best_match=match_candidate
                best_dist=dist

        if best_match: # if found prhase, call my self with exact phrase so first part of main IF can take care of it
            return str_to_ent(whole_str, best_match, tags)

    return result

       
# Test
ents_with_tolerance= str_to_ent("this is New York, Peter ... New York, New York ... Baby","New York",["B-LOC","I-LOC"], 3)
print(f"This should find New York 3 times: {ents_with_tolerance}")

my_tokens=["EU", "rejects", "German", "call"]
my_ner_tags=[3, 0, 7, 0]
sentence = " ".join(my_tokens)
ents = []

for phrase, tags in get_NER_phrases(my_tokens, my_ner_tags):
    ents+=str_to_ent(sentence,phrase,[tagId_to_label(t) for t in tags])
print(my_tokens)
print(f"Exact matches: {ents}")


This should find New York 3 times: [(8, 11, 'B-LOC'), (12, 16, 'I-LOC'), (28, 31, 'B-LOC'), (32, 36, 'I-LOC'), (38, 41, 'B-LOC'), (42, 46, 'I-LOC')]
['EU', 'rejects', 'German', 'call']
Exact matches: [(0, 2, 'B-ORG (3)'), (11, 17, 'B-MISC (7)')]


In [148]:
def entities_to_tokens_and_tags(whole_str, entities, get_NER_tag_id_lambda=None):
    tokens = tokenize(whole_str)
    current_ner_tags=[]
    entity_tokens = [whole_str[ent[0]:ent[1]] for ent in entities]
    if get_NER_tag_id_lambda:
        entity_tags = [get_NER_tag_id_lambda(ent[2]) for ent in entities]
    else:
        entity_tags = [ent[2] for ent in entities]
    for token in tokens:
        if token in entity_tokens:
            i = entity_tokens.index(token)
            current_ner_tags.append(entity_tags[i])
        else:
            current_ner_tags.append(0)
    return (tokens,current_ner_tags)
            
# Test
print(sentence)
print(ents)
print(entities_to_tokens_and_tags(sentence,ents, label_to_tagId ))

EU rejects German call
[(0, 2, 'B-ORG (3)'), (11, 17, 'B-MISC (7)')]
(['EU', 'rejects', 'German', 'call'], [3, 0, 7, 0])


In [149]:

hardcoded_translations = {
    "ENGLISH":"Anglický",
    "Chilean":"Čilský",
    "U.S. OPEN":"Čilský",
    "British-based":"Britský",
    "South":"Juh",
    "CAIRO":"Káhira",
    "Cairo":"Káhira",
    "Republican":"republikánsky",
    "U.S.":"USA",
    "Kurdish":"kurdský",
    "BUCHAREST":"Bukurešť",
    "South African":"Juho afircký",
    "Calcutta":"Kalkata",
    "Japanese":"Japonský",
    "Japanese":"Japonský",
    "ENGLAND":"Anglicko",
    "South Korean":"Juho Kórejský",
    "MAJOR LEAGUE":"HLAVNÁ LIGA",
    "Serb":"Srb",
    "Serb":"Srb",
    "Yeltsin":"Jeľcin",
    "Chechnya":"Čečensko",
    "Turkish":"turecké",
    "DUTCH":"Holandský",
    "C$":"kanadský",
    "A$":"austrálsky",
    "World Cup":"majstrovstvá sveta",
    "Irish":"írsky",
    "U.S.":"USA",
    "Calif.":"Kalifornia",
    "South African":"Juho Africký",
    "South Korean":"Juho Kórejský",
    "Yugoslav":"juhoslovanský",
    "Boris Yeltsin":"Boris Jeľcin",
    "WARSAW":"Varšava",
    "Yasser Arafat":"Jásir Arafat",
    "New Zealand":"Nový Zéland",
    "U.N.":"OSN",
    "Swe":"Švédsko",
    "Japan":"Japonsko",
    "West Bank":"Západný breh Jordánu",
    "NEW DELHI":"Nové Dílí",
    "S. Korea":"Južná Kórea",
    "Sri Lankan":"srílanskí",
    "Saudi Arabia":"Saudský Arábia",
    "Karachi":"Karáčí",
    "PLO":"OOP",
    "COPENHAGEN":"KODAŇ",
    "MUNICH":"Mníchov"

}


def translate_phrase(text):
    if (text in hardcoded_translations):
        return hardcoded_translations[text]
    res =  en_sk_pipeline(text)[0]["generated_text"]
    if res.endswith("Name"): #just a hack because sometimes mdel returns silly results like German -> NemeckýName
        res = res[:-4]
    if "(optional, probably does not need a translation)" in res: # another hack... for example Dynamo Kiev -> Dynamo KyjevCity name (optional, probably does not need a translation) :(
        return text
    return res



translate_phrase("BUCHAREST")

'Bukurešť'

In [150]:
def find_phrase(original_name, full_str):
    try:
        #return " ".join(n[:max(len(n)-2,2)] + "\w{1,4}" if len(n)>3 else n  for n in original_name.split(" "))
        pattern = " ".join(re.escape(n[:max(len(n)-2,2)]) + "\w{1,4}" if len(n)>3 else re.escape(n)  for n in original_name.split(" "))
        for found in re.findall(pattern, full_str,flags=re.IGNORECASE):
            return found
    except Exception as ex:
        print(f"Warning: Error {ex} in {pattern}")    

    
find_phrase("Peter (v) Sagan","Petra (V) Sagana")

'Petra (V) Sagana'

In [151]:
theTranslatedDataSetDF["translated"] = theTranslatedDataSetDF["translated"].apply(lambda text: BeautifulSoup(text).get_text())

/Users/juraj/.local/share/virtualenvs/SK-NER-_-xGSSlM/lib/python3.8/site-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: "." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [152]:
def enumerate_translation(df, include_missing_in_text=True):

    for idx, row in df.iterrows():
        translated = row["translated"] 
        #soup = BeautifulSoup(row["translated"])
        #translated=soup.get_text()
        #translated_tokens = tokenize(row["translated"].lower())
        ents=[]
        missings=[]
        ai_translated_counter=0
        for phrase, tagIds in get_NER_phrases(row["tokens"],row["ner_tags"]):
            tag_labels = [tagId_to_label(tag_id) for tag_id in tagIds]
            found_ents = str_to_ent(translated,phrase,tag_labels)
            if len(found_ents)>0: # this is useful for names for example
                ents+=found_ents
            else:
                if (tagIds[0]==1): #if is persons name, no need to translate
                    translated_phrase=phrase
                else:
                    translated_phrase = translate_phrase(phrase)
                
                found_name = find_phrase(translated_phrase,translated) # try to find names with help of regex
                if found_name:
                    found_ents = str_to_ent(translated,found_name,tag_labels)
                else:
                    found_ents=[]
                
                if len(found_ents)>0:
                    ents+=found_ents
                else:
                    translated_phrase = translate_phrase(phrase)
                    found_ents = str_to_ent(translated,translated_phrase,tag_labels, match_tolerance=int(max(len(translated_phrase),len(phrase))/2))
                    if len(found_ents)>0: # this is useful for names for example
                        ents+=found_ents
                        ai_translated_counter+=1
                    else:
                        missings.append((phrase,tag_labels[0])) #missing can contain the whole phrase, it's easier to read

        if include_missing_in_text:   
            text=translated
            for i,missing in enumerate(missings):
                if (i>0):
                    text+=", "
                else:
                    text=translated  + " // MISSING: " 
                ents.append((len(text),len(text)+len(missing[0]), missing[1])) # (start, end, label)
                text+=missing[0] # add missing tokend to end of text
                
            yield {
                "index":idx,
                "text":text, 
                "entities":ents  , 
                "not-matched":missings,
                "original":row["tokens"],
                "ner_tags":row["ner_tags"],
                "matched_by_ai_translation":ai_translated_counter
                }
        else:
            yield {
                "index":idx,
                "text":translated, 
                "entities":ents, 
                "not-matched":missings, 
                "matched_by_ai_translation":ai_translated_counter}




#tests
x = enumerate_translation(theTranslatedDataSetDF.loc[1:1,:])
y=next(x)
print(y["text"])
print([(e[0],e[1]) for e in y["entities"]])
ewords = [y["text"][e[0]:e[1]] for e in y["entities"]]
print(ewords)
print(y)
 



LONDÝN 1996-08-30
[(0, 6)]
['LONDÝN']
{'index': 1, 'text': 'LONDÝN 1996-08-30', 'entities': [(0, 6, 'B-LOC (5)')], 'not-matched': [], 'original': array(['LONDON', '1996-08-30'], dtype=object), 'ner_tags': array([5, 0]), 'matched_by_ai_translation': 0}


In [153]:


from tqdm import tqdm


def preprocess(tf_to_process, rng_size=None):

    if rng_size is not None:
        tf_to_process=pd.DataFrame(tf_to_process.iloc[0:rng_size,:])

    else:
        tf_to_process=pd.DataFrame(tf_to_process)
        rng_size=len(tf_to_process)
    
    input_queue=enumerate_translation(tf_to_process)
    
    not_matched=[]
    tf_to_process["new_ner_tags"]=None
    tf_to_process["new_tokens"]=None

    for i in tqdm(range(rng_size)):
        item  = next(input_queue)
        if len(item["not-matched"])>0:
            not_matched.append(item)
        else:
            (new_tokens, new_ner_tags) = entities_to_tokens_and_tags(item["text"],item["entities"],label_to_tagId)
            tf_to_process.loc[item["index"]]["new_ner_tags"] =new_ner_tags
            tf_to_process.loc[item["index"]]["new_tokens"] = new_tokens

    
    print(f"not-matched-count: {len(not_matched)}")
    return (tf_to_process,not_matched)

(df_to_be_annotated,not_matched) = preprocess(theTranslatedDataSetDF)

100%|██████████| 3250/3250 [22:40<00:00,  2.39it/s]  

not-matched-count: 290


In [199]:
from IPython.display import display_html

def find_sport_results(tokens):
    if len(tokens)>5:
        for i in range(1,4):
            if not tokens[-i].isnumeric() or len(tokens[-i])>2:
                return False
        exclude_tokens=["Strasbourg","Paris", "Newsroom", "newsroom", "(" ,"Japan","Syria","China","Munich"] # skip this records because they are better if they are translated
        for exclude in exclude_tokens:
            if  exclude in tokens:
                return False
        
        return True
    return False


todo  = df_to_be_annotated[df_to_be_annotated["tokens"].apply(lambda x: find_sport_results(x))]
#display_html(todo)
todo[0:].head(50)


,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
1166,1166,"[Puchon, 1, 1, 0, 1, 0, 4]","[3, 0, 0, 0, 0, 0, 0]",Puchon 1 1 0 1 0 4,"[3, 0, 0, 0, 0, 0, 0]","[Puchon, 1, 1, 0, 1, 0, 4]"
1167,1167,"[Chonan, 1, 0, 0, 5, 4, 3]","[3, 0, 0, 0, 0, 0, 0]",Chonan 1 0 0 5 4 3,"[3, 0, 0, 0, 0, 0, 0]","[Chonan, 1, 0, 0, 5, 4, 3]"
1168,1168,"[Anyang, 0, 2, 0, 5, 5, 2]","[3, 0, 0, 0, 0, 0, 0]",Anyang 0 2 0 5 5 2,"[3, 0, 0, 0, 0, 0, 0]","[Anyang, 0, 2, 0, 5, 5, 2]"
1169,1169,"[Suwon, 0, 2, 0, 3, 3, 2]","[3, 0, 0, 0, 0, 0, 0]",Suwon 0 2 0 3 3 2,"[3, 0, 0, 0, 0, 0, 0]","[Suwon, 0, 2, 0, 3, 3, 2]"
1170,1170,"[Pohang, 0, 1, 0, 3, 3, 1]","[3, 0, 0, 0, 0, 0, 0]",Pohang 0 1 0 3 3 1,"[3, 0, 0, 0, 0, 0, 0]","[Pohang, 0, 1, 0, 3, 3, 1]"
1171,1171,"[Pusan, 0, 1, 0, 0, 2, 1]","[3, 0, 0, 0, 0, 0, 0]",Pusan 0 1 0 0 2 1,"[3, 0, 0, 0, 0, 0, 0]","[Pusan, 0, 1, 0, 0, 2, 1]"
1172,1172,"[Chonnam, 0, 1, 1, 5, 6, 1]","[3, 0, 0, 0, 0, 0, 0]",Chonnam 0 1 1 5 6 1,"[3, 0, 0, 0, 0, 0, 0]","[Chonnam, 0, 1, 1, 5, 6, 1]"
1173,1173,"[Ulsan, 0, 0, 1, 4, 5, 0]","[3, 0, 0, 0, 0, 0, 0]",Ulsan 0 0 1 4 5 0,"[3, 0, 0, 0, 0, 0, 0]","[Ulsan, 0, 0, 1, 4, 5, 0]"
1174,1174,"[Chonbuk, 0, 0, 0, 0, 0, 0]","[3, 0, 0, 0, 0, 0, 0]",Chonbuk 0 0 0 0 0 0,"[3, 0, 0, 0, 0, 0, 0]","[Chonbuk, 0, 0, 0, 0, 0, 0]"
3755,3755,"[Becej, 3, 2, 1, 0, 5, 1, 7]","[3, 0, 0, 0, 0, 0, 0, 0]",Bečej 3 2 1 0 5 1 7,"[3, 0, 0, 0, 0, 0, 0, 0]","[Becej, 3, 2, 1, 0, 5, 1, 7]"


In [206]:

index_to_fix = df_to_be_annotated[df_to_be_annotated["new_ner_tags"].apply(lambda x: sum([1 for t in x if type(t) is str ])>0)].index
df_to_be_annotated.loc[index_to_fix]

,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
4084,4084,"[Bayern, Munich, 2, 1, 1, 0, 3, 2, 4]","[3, 4, 0, 0, 0, 0, 0, 0, 0]",Bayern Mníchov 2 1 1 0 3 2 4,"[Bayern, Munich, 2, 1, 1, 0, 3, 2, 4]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
4086,4086,"[1860, Munich, 3, 1, 0, 2, 3, 5, 3]","[3, 4, 0, 0, 0, 0, 0, 0, 0]",1860 Mníchov 3 1 0 2 3 5 3,"[1860, Munich, 3, 1, 0, 2, 3, 5, 3]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
5540,5540,"[Bayern, Munich, 3, 2, 1, 0, 7, 2, 7]","[3, 4, 0, 0, 0, 0, 0, 0, 0]",Bayern Mníchov 3 2 1 0 7 2 7,"[Bayern, Munich, 3, 2, 1, 0, 7, 2, 7]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
5548,5548,"[1860, Munich, 3, 1, 0, 2, 3, 5, 3]","[3, 4, 0, 0, 0, 0, 0, 0, 0]",1860 Mníchov 3 1 0 2 3 5 3,"[1860, Munich, 3, 1, 0, 2, 3, 5, 3]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
8702,8702,"[Bayern, Munich, 3, 2, 1, 0, 7, 2, 7]","[3, 4, 0, 0, 0, 0, 0, 0, 0]",Bayern Mníchov 3 2 1 0 7 2 7,"[Bayern, Munich, 3, 2, 1, 0, 7, 2, 7]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
8709,8709,"[1860, Munich, 3, 1, 0, 2, 3, 5, 3]","[3, 4, 0, 0, 0, 0, 0, 0, 0]",1860 Mníchov 3 1 0 2 3 5 3,"[1860, Munich, 3, 1, 0, 2, 3, 5, 3]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
10712,10712,"[Bayern, Munich, 4, 3, 1, 0, 11, 4, 10]","[3, 4, 0, 0, 0, 0, 0, 0, 0]",Bayern Mníchov 4 3 1 0 11 4 10,"[Bayern, Munich, 4, 3, 1, 0, 11, 4, 10]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
10722,10722,"[Munich, 1860, 4, 1, 1, 2, 3, 5, 4]","[3, 4, 0, 0, 0, 0, 0, 0, 0]",Mníchov 1860 4 1 1 2 3 5 4,"[Munich, 1860, 4, 1, 1, 2, 3, 5, 4]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"


In [193]:
#comit to this... sports result we don't translate

df_to_be_annotated.at[todo.index,"new_ner_tags"] = todo["ner_tags"]
df_to_be_annotated.at[todo.index,"new_tokens"] = todo["tokens"]
display_html(df_to_be_annotated.loc[df_to_be_annotated["new_tokens"].isnull()][df_to_be_annotated["tokens"].apply(lambda x: find_sport_results(x))])

/var/folders/h5/y7fcd0xd4x71d0639gs2z5c00000gp/T/ipykernel_41491/902954438.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display_html(df_to_be_annotated.loc[df_to_be_annotated["new_tokens"].isnull()][df_to_be_annotated["tokens"].apply(lambda x: find_sport_results(x))])


,id,tokens,ner_tags,translated,new_ner_tags,new_tokens


In [194]:
#review what's left
pd.set_option('display.max_colwidth', None)
todo  = df_to_be_annotated.loc[df_to_be_annotated["new_tokens"].isnull()]
todo 

,id,tokens,ner_tags,translated,new_ner_tags,new_tokens


In [157]:
not_matched_df = pd.DataFrame(not_matched)

In [167]:
#exclude sport results that have been replaced before
not_matched_df = not_matched_df[~not_matched_df["original"].apply(lambda x: find_sport_results(x))]

In [466]:
# run this if additional translations were found
""" 
todo  = df_to_be_annotated.loc[df_to_be_annotated["new_tokens"].isnull()]
todo = todo[df_to_be_annotated["tokens"].apply(lambda x: "Saudi" in x)]
display_html(todo.tail(50))

(not_annotated_yet,not_matched) = preprocess(todo)
df_to_be_annotated.at[not_annotated_yet.index,"new_ner_tags"] = not_annotated_yet["new_ner_tags"]
df_to_be_annotated.at[not_annotated_yet.index,"new_tokens"] = not_annotated_yet["new_tokens"]
display_html(todo[todo["new_tokens"].isnull()]) 
"""

/var/folders/h5/y7fcd0xd4x71d0639gs2z5c00000gp/T/ipykernel_11885/2919174826.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  todo = todo[df_to_be_annotated["tokens"].apply(lambda x: "Saudi" in x)]


,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
7533,7533,"[U.S., Senate, Intelligence, Committee, chairman, Arlen, Specter, ,, who, has, questioned, whether, Defence, Secretary, William, Perry, should, resign, over, the, latest, bombing, in, Saudi, Arabia, ,, met, Saudi, officials, on, Monday, during, a, brief, visit, to, the, kingdom, .]","[3, 4, 4, 4, 0, 1, 2, 0, 0, 0, 0, 0, 3, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Predseda amerického senátneho výboru pre spravodajské služby Arlen Spectre, ktorý sa pýtal, či by mal minister obrany William Perry odstúpiť pre posledný bombový útok v Saudskej Arábii, sa v pondelok stretol so saudskými predstaviteľmi počas krátkej návštevy kráľovstva.",None,None
7534,7534,"[A, U.S., embassy, spokesman, in, Riyadh, said, Specter, ,, who, arrived, from, neighbouring, Oman, on, Sunday, and, left, on, Monday, ,, had, talks, with, Saudi, and, American, officials, in, Dhahran, ,, where, 19, U.S., airmen, were, killed, by, a, fuel, truck, bomb, on, June, 25, ,, and, Riyadh, .]","[0, 5, 0, 0, 0, 5, 0, 1, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 5, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0]","Hovorca amerického veľvyslanectva v Rijáde uviedol, že Spectre, ktorý pricestoval zo susedného Ománu v nedeľu a odišiel v pondelok, rokoval so saudskoarabskými a americkými predstaviteľmi v Dhahráne, kde 25. júna zahynulo 19 amerických letcov bombou v nákladnom aute, ako aj v Rijáde.",None,None
7535,7535,"[Specter, met, Crown, Prince, Abdullah, and, Minister, of, Defence, and, Aviation, Prince, Sultan, in, Jeddah, ,, Saudi, state, television, and, the, official, Saudi, Press, Agency, reported, .]","[1, 0, 0, 1, 2, 0, 0, 0, 3, 4, 4, 0, 1, 0, 5, 0, 7, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0]","Spectre sa stretol s korunným princom Abdullahom a ministrom obrany a letectva princom Sultanom v Jeddahu, informovala saudská štátna televízia a oficiálna saudská tlačová agentúra.",None,None
13179,13179,"[EU, barley, sale, worth, $, 145, /, T, ,, for, Saudi, -, sources, .]","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]",Predaj jačmeňa v EÚ v hodnote 145 USD / T pre saudské zdroje .,None,None
13414,13414,"[New, U.S., ambassador, arrives, in, Saudi, Arabia, .]","[0, 5, 0, 0, 0, 5, 6, 0]",Nový americký veľvyslanec prichádza do Saudskej Arábie.,None,None
13416,13416,"[Washington, 's, new, ambassador, to, Saudi, Arabia, ,, Wyche, Fowler, ,, arrived, in, the, kingdom, to, take, up, his, post, ,, the, U.S., embassy, in, Riyadh, said, on, Thursday, .]","[5, 0, 0, 0, 0, 5, 6, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0]","Nový veľvyslanec Washingtonu v Saudskej Arábii, Wyche Fowler, pricestoval do kráľovstva, aby prevzal svoj post, oznámilo vo štvrtok americké veľvyslanectvo v Rijáde.",None,None


100%|██████████| 6/6 [00:11<00:00,  1.89s/it]

not-matched-count: 6


,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
7533,7533,"[U.S., Senate, Intelligence, Committee, chairman, Arlen, Specter, ,, who, has, questioned, whether, Defence, Secretary, William, Perry, should, resign, over, the, latest, bombing, in, Saudi, Arabia, ,, met, Saudi, officials, on, Monday, during, a, brief, visit, to, the, kingdom, .]","[3, 4, 4, 4, 0, 1, 2, 0, 0, 0, 0, 0, 3, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Predseda amerického senátneho výboru pre spravodajské služby Arlen Spectre, ktorý sa pýtal, či by mal minister obrany William Perry odstúpiť pre posledný bombový útok v Saudskej Arábii, sa v pondelok stretol so saudskými predstaviteľmi počas krátkej návštevy kráľovstva.",None,None
7534,7534,"[A, U.S., embassy, spokesman, in, Riyadh, said, Specter, ,, who, arrived, from, neighbouring, Oman, on, Sunday, and, left, on, Monday, ,, had, talks, with, Saudi, and, American, officials, in, Dhahran, ,, where, 19, U.S., airmen, were, killed, by, a, fuel, truck, bomb, on, June, 25, ,, and, Riyadh, .]","[0, 5, 0, 0, 0, 5, 0, 1, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 5, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0]","Hovorca amerického veľvyslanectva v Rijáde uviedol, že Spectre, ktorý pricestoval zo susedného Ománu v nedeľu a odišiel v pondelok, rokoval so saudskoarabskými a americkými predstaviteľmi v Dhahráne, kde 25. júna zahynulo 19 amerických letcov bombou v nákladnom aute, ako aj v Rijáde.",None,None
7535,7535,"[Specter, met, Crown, Prince, Abdullah, and, Minister, of, Defence, and, Aviation, Prince, Sultan, in, Jeddah, ,, Saudi, state, television, and, the, official, Saudi, Press, Agency, reported, .]","[1, 0, 0, 1, 2, 0, 0, 0, 3, 4, 4, 0, 1, 0, 5, 0, 7, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0]","Spectre sa stretol s korunným princom Abdullahom a ministrom obrany a letectva princom Sultanom v Jeddahu, informovala saudská štátna televízia a oficiálna saudská tlačová agentúra.",None,None
13179,13179,"[EU, barley, sale, worth, $, 145, /, T, ,, for, Saudi, -, sources, .]","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]",Predaj jačmeňa v EÚ v hodnote 145 USD / T pre saudské zdroje .,None,None
13414,13414,"[New, U.S., ambassador, arrives, in, Saudi, Arabia, .]","[0, 5, 0, 0, 0, 5, 6, 0]",Nový americký veľvyslanec prichádza do Saudskej Arábie.,None,None
13416,13416,"[Washington, 's, new, ambassador, to, Saudi, Arabia, ,, Wyche, Fowler, ,, arrived, in, the, kingdom, to, take, up, his, post, ,, the, U.S., embassy, in, Riyadh, said, on, Thursday, .]","[5, 0, 0, 0, 0, 5, 6, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0]","Nový veľvyslanec Washingtonu v Saudskej Arábii, Wyche Fowler, pricestoval do kráľovstva, aby prevzal svoj post, oznámilo vo štvrtok americké veľvyslanectvo v Rijáde.",None,None


In [98]:
x = pd.DataFrame({"phrase":pd.DataFrame(not_matched)["not-matched"].explode().apply(lambda x: x[0])}).groupby(by="phrase",as_index=False).size().sort_values(by="size").tail(50)
#x["translated"] = x["phrase"].apply(lambda x: translate_phrase(x))
x

,phrase,size
40,Democratic Left Alliance,1
41,Department of Corrections,1
56,"Goldman , Sachs",1
57,Gulf,1
59,House of Representatives,1
42,Department of Transport,1
17,Boutros-Ghali,2
78,MUNICH,2
143,WEST INDIES,2
130,U.S. Municipal Desk,2


In [168]:
df_to_be_annotated.to_pickle(f"df_to_be_annotated_{split_name}.picle")
df_to_be_annotated

,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
0,0,"[CRICKET, -, LEICESTERSHIRE, TAKE, OVER, AT, TOP, AFTER, INNINGS, VICTORY, .]","[0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]",KRIKET - LEICESTERSHIRE prevzatie NA VRCHOLE PO VÍŤAZSTVE NA ZMENE .,"[0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]","[KRIKET, -, LEICESTERSHIRE, prevzatie, NA, VRCHOLE, PO, VÍŤAZSTVE, NA, ZMENE, .]"
1,1,"[LONDON, 1996-08-30]","[5, 0]",LONDÝN 1996-08-30,"[5, 0, 0, 0, 0, 0]","[LONDÝN, 1996, -, 08, -, 30]"
2,2,"[West, Indian, all-rounder, Phil, Simmons, took, four, for, 38, on, Friday, as, Leicestershire, beat, Somerset, by, an, innings, and, 39, runs, in, two, days, to, take, over, at, the, head, of, the, county, championship, .]","[7, 8, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Západoindický univerzál Phil Simmons získal v piatok štyri za 38, keď Leicestershire porazil Somerset o zmenu a 39 jázd za dva dni, aby prevzal vedenie na čele krajského šampionátu.",None,None
3,3,"[Their, stay, on, top, ,, though, ,, may, be, short-lived, as, title, rivals, Essex, ,, Derbyshire, and, Surrey, all, closed, in, on, victory, while, Kent, made, up, for, lost, time, in, their, rain-affected, match, against, Nottinghamshire, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0]","Ich pobyt na vrchole však môže trvať len krátko, pretože súperi o titul Essex, Derbyshire a Surrey dosiahli víťazstvo, zatiaľ čo Kent dohnal stratený čas v zápase s Nottinghamshire postihnutým dažďom.","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0]","[Ich, pobyt, na, vrchole, však, môže, trvať, len, krátko, ,, pretože, súperi, o, titul, Essex, ,, Derbyshire, a, Surrey, dosiahli, víťazstvo, ,, zatiaľ, čo, Kent, dohnal, stratený, čas, v, zápase, s, Nottinghamshire, postihnutým, dažďom, .]"
4,4,"[After, bowling, Somerset, out, for, 83, on, the, opening, morning, at, Grace, Road, ,, Leicestershire, extended, their, first, innings, by, 94, runs, before, being, bowled, out, for, 296, with, England, discard, Andy, Caddick, taking, three, for, 83, .]","[0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0]","Po bowlingu Somerset za 83 bodov v úvodné ráno na Grace Road, Leicestershire predĺžil svoje prvé smeny o 94 behov, než bol odhodený za 296, pričom Anglicko vyradilo Andyho Caddicka a tri za 83.","[0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0]","[Po, bowlingu, Somerset, za, 83, bodov, v, úvodné, ráno, na, Grace, Road, ,, Leicestershire, predĺžil, svoje, prvé, smeny, o, 94, behov, ,, než, bol, odhodený, za, 296, ,, pričom, Anglicko, vyradilo, Andyho, Caddicka, a, tri, za, 83, .]"
...,...,...,...,...,...,...
3245,3245,"[But, the, prices, may, move, in, a, close, range, following, a, continued, market, uptrend, ., ""]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Ceny sa však môžu pohybovať v blízkom rozmedzí po pokračujúcom vzostupnom trende trhu. ""","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Ceny, sa, však, môžu, pohybovať, v, blízkom, rozmedzí, po, pokračujúcom, vzostupnom, trende, trhu, ., ""]"
3246,3246,"[Brokers, said, blue, chips, like, IDLC, ,, Bangladesh, Lamps, ,, Chittagong, Cement, and, Atlas, Bangladesh, were, expected, to, rise, .]","[0, 0, 0, 0, 0, 3, 0, 3, 4, 0, 3, 4, 0, 3, 4, 0, 0, 0, 0, 0]","Makléri uviedli, že blue chips ako IDLC, Bangladesh Lamps, Chittagong Cement a Atlas Bangladesh sa očakáva nárast.","[0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 4, 0, 3, 4, 0, 3, 3, 0, 0, 0, 0]","[Makléri, uviedli, ,, že, blue, chips, ako, IDLC, ,, Bangladesh, Lamps, ,, Chittagong, Cement, a, Atlas, Bangladesh, sa, očakáva, nárast, .]"
3247,3247,"[They, said, there, was, still, demand, for, blue, chips, in, engineering, sector, despite, their, persistent, rise, over, the, past, several, sessions, .]","[

In [169]:
not_matched_df = pd.DataFrame(not_matched)
not_matched_df.to_pickle("not_matched_df.picle")
not_matched_df

,index,text,entities,not-matched,original,ner_tags,matched_by_ai_translation
0,2,"Západoindický univerzál Phil Simmons získal v piatok štyri za 38, keď Leicestershire porazil Somerset o zmenu a 39 jázd za dva dni, aby prevzal vedenie na čele krajského šampionátu. // MISSING: West Indian","[(24, 28, B-PER (1)), (29, 36, I-PER (2)), (70, 84, B-ORG (3)), (93, 101, B-ORG (3)), (194, 205, B-MISC (7))]","[(West Indian, B-MISC (7))]","[West, Indian, all-rounder, Phil, Simmons, took, four, for, 38, on, Friday, as, Leicestershire, beat, Somerset, by, an, innings, and, 39, runs, in, two, days, to, take, over, at, the, head, of, the, county, championship, .]","[7, 8, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
1,16,SKÓRE NA MAJSTROVSTVÁCH V KRIKETE - ANGLICKOM COUNTY. // MISSING: ENGLISH COUNTY CHAMPIONSHIP,"[(66, 93, B-MISC (7))]","[(ENGLISH COUNTY CHAMPIONSHIP, B-MISC (7))]","[CRICKET, -, ENGLISH, COUNTY, CHAMPIONSHIP, SCORES, .]","[0, 0, 7, 8, 8, 0, 0]",0
2,53,"25. máj Tretí jednodňový medzinárodný (v Lord's, Londýn) // MISSING: Lord 's","[(49, 55, B-LOC (5)), (69, 76, B-LOC (5))]","[(Lord 's, B-LOC (5))]","[May, 25, Third, one-day, international, (, at, Lord, 's, ,, London, )]","[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 0]",0
3,66,9. júla proti Menším župám XI // MISSING: Minor Counties XI,"[(42, 59, B-ORG (3))]","[(Minor Counties XI, B-ORG (3))]","[July, 9, v, Minor, Counties, XI]","[0, 0, 0, 3, 4, 4]",0
4,76,FUTBALOVÝ STROJ MENOVANÝ AKO ANGLICKÝ KAPITÁN. // MISSING: SHEARER,"[(29, 37, B-LOC (5)), (59, 66, B-PER (1))]","[(SHEARER, B-PER (1))]","[SOCCER, -, SHEARER, NAMED, AS, ENGLAND, CAPTAIN, .]","[0, 0, 1, 0, 0, 5, 0, 0]",0
...,...,...,...,...,...,...,...
285,3219,"Hoci sa má prepustenie v Británii začať až začiatkom budúceho roka, niektorí politici už povedali, že sa obávajú, že to rozdúcha sektárske napätie v Britmi ovládanom Severnom Írsku. // MISSING: British-ruled","[(25, 33, B-LOC (5)), (166, 174, B-LOC (5)), (175, 180, I-LOC (6)), (194, 207, B-MISC (7))]","[(British-ruled, B-MISC (7))]","[Although, not, due, for, release, in, Britain, until, early, next, year, ,, some, politicians, have, already, said, they, feared, it, would, fan, sectarian, tensions, in, British-ruled, Northern, Ireland, .]","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 5, 6, 0]",0
286,3221,"""Film nešetrí ani Írov, ani Britov v zobrazení divokosti tej doby,"" uviedol Jordan vo vyhlásení zverejnenom Warner Bros. "" // MISSING: Irish","[(28, 34, B-MISC (7)), (76, 82, B-PER (1)), (108, 114, B-ORG (3)), (115, 119, I-ORG (4)), (135, 140, B-MISC (7))]","[(Irish, B-MISC (7))]","["", The, film, spares, neither, the, Irish, nor, the, British, in, its, depiction, of, the, savagery, of, the, time, ,, "", Jordan, said, in, a, statement, released, by, Warner, Bros, ., ""]","[0, 0, 0, 0, 0, 0, 7, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0]",1
287,3225,"""Vyvinul techniky partizánskeho boja, ktoré neskôr skopírovali hnutia za nezávislosť po celom svete, od Mao Ce-Tunga v Číne až po Jiczaka Shamira v Izraeli,"" povedal Jordan. // MISSING: Mao Tse-Tung","[(119, 123, B-LOC (5)), (130, 137, B-PER (1)), (138, 145, I-PER (2)), (148, 155, B-LOC (5)), (166, 172, B-PER (1)), (186, 198, B-PER (1))]","[(Mao Tse-Tung, B-PER (1))]","["", He, developed, techniques, of, guerilla, warfare, later, copied, by, independence, movements, around, the, world, ,, from, Mao, Tse-Tung, in, China, to, Yitzak, Shamir, in, Israel, ,, "", Jordan, said, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 5, 0, 1, 2, 0, 5, 0, 0, 1, 0, 0]",1
288,3241,"Akcie na burze v Dháke (DSE) môžu zostať stabilné, keďže sa očakáva, že malí investori sa zacieliť hlavne na blue chips, zatiaľ čo zahraniční investori budú radšej držať bokom, keď sa trh po víkende moslimského piatku znovu otvorí, uviedli makléri. // MISSING: Dhaka Stock Exchange","[(24, 27, B-ORG (3)), (199, 207, B-MISC (7)),

# Load df_to_be_annotated
# ... here...

In [383]:

df_to_be_annotated= pd.read_pickle(f"df_to_be_annotated_{split_name}.picle")
not_matched_df= pd.read_pickle("not_matched_df.picle")
df_to_be_annotated

,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
0,0,"[EU, rejects, German, call, to, boycott, British, lamb, .]","[3, 0, 7, 0, 0, 0, 7, 0, 0]",EÚ odmieta nemeckú výzvu na bojkot britského jahňacieho mäsa.,"[3, 0, 7, 0, 0, 0, 7, 0, 0]","[EÚ, odmieta, nemeckú, výzvu, na, bojkot, britského, jahňacieho, mäsa.]"
1,1,"[Peter, Blackburn]","[1, 2]",Peter Blackburn,"[1, 2]","[Peter, Blackburn]"
2,2,"[BRUSSELS, 1996-08-22]","[5, 0]",BRUSEL 1996-08-22,"[5, 0]","[BRUSEL, 1996-08-22]"
3,3,"[The, European, Commission, said, on, Thursday, it, disagreed, with, German, advice, to, consumers, to, shun, British, lamb, until, scientists, determine, whether, mad, cow, disease, can, be, transmitted, to, sheep, .]","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Európska komisia vo štvrtok uviedla, že nesúhlasí s nemeckou radou spotrebiteľom, aby sa vyhýbali britskej jahňacine, kým vedci nezistia, či sa choroba šialených kráv môže preniesť na ovce.","[3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Európska, komisia, vo, štvrtok, uviedla,, že, nesúhlasí, s, nemeckou, radou, spotrebiteľom,, aby, sa, vyhýbali, britskej, jahňacine,, kým, vedci, nezistia,, či, sa, choroba, šialených, kráv, môže, preniesť, na, ovce.]"
4,4,"[Germany, 's, representative, to, the, European, Union, 's, veterinary, committee, Werner, Zwingmann, said, on, Wednesday, consumers, should, buy, sheepmeat, from, countries, other, than, Britain, until, the, scientific, advice, was, clearer, .]","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]","Zástupca Nemecka vo veterinárnom výbore Európskej únie Werner Zwingmann v stredu povedal, že spotrebitelia by mali kupovať ovčie mäso z iných krajín ako z Británie, kým nebudú jasné vedecké odporúčania.","[0, 5, 0, 0, 0, 3, 4, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0]","[Zástupca, Nemecka, vo, veterinárnom, výbore, Európskej, únie, Werner, Zwingmann, v, stredu, povedal,, že, spotrebitelia, by, mali, kupovať, ovčie, mäso, z, iných, krajín, ako, z, Británie,, kým, nebudú, jasné, vedecké, odporúčania.]"
...,...,...,...,...,...,...
14036,14036,"[on, Friday, :]","[0, 0, 0]",v piatok :,"[0, 0, 0]","[v, piatok, :]"
14037,14037,"[Division, two]","[0, 0]",Divízia dva,"[0, 0]","[Divízia, dva]"
14038,14038,"[Plymouth, 2, Preston, 1]","[3, 0, 3, 0]",Plymouth 2 Preston 1,"[3, 0, 3, 0]","[Plymouth, 2, Preston, 1]"
14039,14039,"[Division, three]","[0, 0]",Divízia tri,"[0, 0]","[Divízia, tri]"


In [59]:

import ipyannotations.text
widget = ipyannotations.text.TextTagger(classes=ner_tags)
labels=[]

input_queue=not_matched
current = {}
def store_annotations(entity_annotation):
    labels.append(entity_annotation)
    (new_tokens, new_ner_tags) = entities_to_tokens_and_tags(current["text"],entity_annotation,label_to_tagId)
    df_to_be_annotated.loc[current["index"]]["new_ner_tags"] =new_ner_tags[:len(new_tokens)]
    df_to_be_annotated.loc[current["index"]]["new_tokens"] = new_tokens

    try:
        next_document(input_queue)
    except StopIteration:
        print("Finished.")

def next_document(enumerator, skip_if_nothing_missing=True):
    item = next(enumerator)
    while (len(item["not-matched"])==0 and skip_if_nothing_missing):
        labels.append(item["entities"])
        item = next(enumerator)
    current=item
    print(item["entities"])
    widget.display(item["text"])
    widget.data = item["entities"]

widget.on_submit(store_annotations)

In [69]:


# start the iterations:
next_document(input_queue)
widget

StopIteration: 

In [215]:
#save anotated data
df_to_be_annotated.to_pickle(f"df_annotated_{split_name}.picle")


In [284]:
df_annotated= pd.read_pickle(f"df_annotated_{split_name}.picle")
df_annotated

,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
0,0,"[EU, rejects, German, call, to, boycott, British, lamb, .]","[3, 0, 7, 0, 0, 0, 7, 0, 0]",EÚ odmieta nemeckú výzvu na bojkot britského jahňacieho mäsa.,"[3, 0, 7, 0, 0, 0, 7, 0, 0]","[EÚ, odmieta, nemeckú, výzvu, na, bojkot, britského, jahňacieho, mäsa.]"
1,1,"[Peter, Blackburn]","[1, 2]",Peter Blackburn,"[1, 2]","[Peter, Blackburn]"
2,2,"[BRUSSELS, 1996-08-22]","[5, 0]",BRUSEL 1996-08-22,"[5, 0]","[BRUSEL, 1996-08-22]"
3,3,"[The, European, Commission, said, on, Thursday, it, disagreed, with, German, advice, to, consumers, to, shun, British, lamb, until, scientists, determine, whether, mad, cow, disease, can, be, transmitted, to, sheep, .]","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Európska komisia vo štvrtok uviedla, že nesúhlasí s nemeckou radou spotrebiteľom, aby sa vyhýbali britskej jahňacine, kým vedci nezistia, či sa choroba šialených kráv môže preniesť na ovce.","[3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Európska, komisia, vo, štvrtok, uviedla,, že, nesúhlasí, s, nemeckou, radou, spotrebiteľom,, aby, sa, vyhýbali, britskej, jahňacine,, kým, vedci, nezistia,, či, sa, choroba, šialených, kráv, môže, preniesť, na, ovce.]"
4,4,"[Germany, 's, representative, to, the, European, Union, 's, veterinary, committee, Werner, Zwingmann, said, on, Wednesday, consumers, should, buy, sheepmeat, from, countries, other, than, Britain, until, the, scientific, advice, was, clearer, .]","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]","Zástupca Nemecka vo veterinárnom výbore Európskej únie Werner Zwingmann v stredu povedal, že spotrebitelia by mali kupovať ovčie mäso z iných krajín ako z Británie, kým nebudú jasné vedecké odporúčania.","[0, 5, 0, 0, 0, 3, 4, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0]","[Zástupca, Nemecka, vo, veterinárnom, výbore, Európskej, únie, Werner, Zwingmann, v, stredu, povedal,, že, spotrebitelia, by, mali, kupovať, ovčie, mäso, z, iných, krajín, ako, z, Británie,, kým, nebudú, jasné, vedecké, odporúčania.]"
...,...,...,...,...,...,...
14036,14036,"[on, Friday, :]","[0, 0, 0]",v piatok :,"[0, 0, 0]","[v, piatok, :]"
14037,14037,"[Division, two]","[0, 0]",Divízia dva,"[0, 0]","[Divízia, dva]"
14038,14038,"[Plymouth, 2, Preston, 1]","[3, 0, 3, 0]",Plymouth 2 Preston 1,"[3, 0, 3, 0]","[Plymouth, 2, Preston, 1]"
14039,14039,"[Division, three]","[0, 0]",Divízia tri,"[0, 0]","[Divízia, tri]"


In [217]:
#this shuld be empty
df_annotated[df_annotated["new_ner_tags"].isnull()]

,id,tokens,ner_tags,translated,new_ner_tags,new_tokens


In [290]:
#finds all the records with tag greater then 8 (it TODO-FIX tag... so these are the records marked as need to fix after annotation)
print(ner_tags)
print(ner_tags[8:])
df_annotated[df_annotated["new_ner_tags"].apply(lambda x: any([1 for i in x if i>8]))] 

['B-PER (1)', 'I-PER (2)', 'B-ORG (3)', 'I-ORG (4)', 'B-LOC (5)', 'I-LOC (6)', 'B-MISC (7)', 'I-MISC (8)', 'TODO-FIX']
['TODO-FIX']


,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
2065,2065,"[Promising, 10th-ranked, American, Chanda, Rubin, has, pulled, out, of, the, U.S., Open, Tennis, Championships, with, a, wrist, injury, ,, tournament, officials, announced, .]","[0, 0, 7, 1, 2, 0, 0, 0, 0, 0, 7, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Sľubná desiata Američanka Chanda Rubinová odstúpila z turnaja US Open pre zranenie zápästia, oznámili predstavitelia turnaja.","[0, 0, 7, 1, 2, 0, 0, 0, 8, 9, 0, 0, 0, 0, 0, 0, 0, 0]","[Sľubná, desiata, Američanka, Chanda, Rubinová, odstúpila, z, turnaja, US, Open, pre, zranenie, zápästia, ,, oznámili, predstavitelia, turnaja, .]"


In [291]:
#fix it
#df_annotated.at[2065,"new_ner_tags"] = [0, 0, 7, 1, 2, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0]

In [218]:
#todo: Clean broken annotations ...

def fill_gaps_in_anotation(ner_tags, return_none_if_nothing_to_repair=False):
    iterator = iter(enumerate(ner_tags))
    filled_ner_tags=[]
    was_filled=False
    max_i = len(ner_tags)-1
    B_tag=None
    for i,tag in iterator:
        
        if tag % 2==1: #is beggining
            B_tag=tag    
        elif tag==0 and i>0 and i<max_i and B_tag is not None and ner_tags[i+1]==B_tag+1: #if previous tag was B-tag and next tag is I- version of that B-tag
            filled_ner_tags.append(ner_tags[i+1])
            was_filled=True
            continue
        elif tag==0 :
            B_tag=None

        filled_ner_tags.append(tag)

    if not was_filled and return_none_if_nothing_to_repair:
        return None
    else:
        return filled_ner_tags

todo = df_annotated[df_annotated["new_ner_tags"].apply(lambda x: fill_gaps_in_anotation(x, return_none_if_nothing_to_repair=True)).notnull()]
todo

,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
32,32,"[State, media, quoted, China, 's, top, negotiator, with, Taipei, ,, Tang, Shubei, ,, as, telling, a, visiting, group, from, Taiwan, on, Wednesday, that, it, was, time, for, the, rivals, to, hold, political, talks, .]","[0, 0, 0, 5, 0, 0, 0, 0, 5, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Štátne médiá citovali hlavného čínskeho vyjednávača s Tchaj-pejom Tang Shubeiho, ktorý v stredu povedal návšteve z Taiwanu, že je čas, aby rivali viedli politické rozhovory.","[0, 0, 0, 0, 7, 0, 0, 5, 0, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Štátne, médiá, citovali, hlavného, čínskeho, vyjednávača, s, Tchaj, -, pejom, Tang, Shubeiho, ,, ktorý, v, stredu, povedal, návšteve, z, Taiwanu, ,, že, je, čas, ,, aby, rivali, viedli, politické, rozhovory, .]"
40,40,"[Consultations, should, be, held, to, set, the, time, and, format, of, the, talks, ,, the, official, Xinhua, news, agency, quoted, Tang, Shubei, ,, executive, vice, chairman, of, the, Association, for, Relations, Across, the, Taiwan, Straits, ,, as, saying, late, on, Wednesday, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0]","Mali by sa uskutočniť konzultácie, aby sa určil čas a formát rozhovorov, citovala oficiálna tlačová agentúra Xinhua v stredu neskoro večer Tang Shubei, výkonný podpredseda Asociácie pre vzťahy naprieč Taiwanským prielivom.","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2, 0, 0, 0, 3, 4, 4, 0, 4, 4, 0]","[Mali, by, sa, uskutočniť, konzultácie, ,, aby, sa, určil, čas, a, formát, rozhovorov, ,, citovala, oficiálna, tlačová, agentúra, Xinhua, v, stredu, neskoro, večer, Tang, Shubei, ,, výkonný, podpredseda, Asociácie, pre, vzťahy, naprieč, Taiwanským, prielivom, .]"
186,186,"[The, Russians, ,, who, said, they, overpowered, the, guards, --, two, armed, with, Kalashnikov, automatic, rifles, --, while, doing, regular, maintenance, work, on, their, Ilyushin, 76, cargo, plane, last, Friday, ,, left, the, UAE, capital, Abu, Dhabi, for, home, on, Sunday, .]","[0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 5, 6, 0, 0, 0, 0, 0]","Rusi, ktorí minulý piatok povedali, že premohli stráže – dvoch vyzbrojených automatickými puškami Kalašnikov – pri pravidelnej údržbe ich nákladného lietadla Iľjušin 76, v nedeľu odišli z hlavného mesta Spojených arabských emirátov Abú Zabí domov.","[7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 6, 5, 6, 0, 0]","[Rusi, ,, ktorí, minulý, piatok, povedali, ,, že, premohli, stráže, –, dvoch, vyzbrojených, automatickými, puškami, Kalašnikov, –, pri, pravidelnej, údržbe, ich, nákladného, lietadla, Iľjušin, 76, ,, v, nedeľu, odišli, z, hlavného, mesta, Spojených, arabských, emirátov, Abú, Zabí, domov, .]"
242,242,"[--, E., Auchard, ,, Wall, Street, bureau, ,, 212-859-1736]","[0, 1, 2, 0, 3, 4, 4, 0, 0]","-- E. Auchard , kancelária Wall Street , 212-859-1736","[0, 0, 1, 0, 2, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0]","[-, -, E, ., Auchard, ,, kancelária, Wall, Street, ,, 212, -, 859, -, 1736]"
1127,1127,"[Sanchez, Vicario, ,, runner-up, to, Graf, at, the, French, Open, and, Wimbledon, ,, begins, play, against, a, qualifier, in, a, quarter, of, the, draw, that, includes, young, talent, Martina, Hingis, ,, the, 16th, seed, ,, before, a, probable, quarter-final, clash, with, seventh-seeded, veteran, Jana, Novotna, .]","[1, 2, 0, 0, 0, 1, 0, 0, 7, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0]","Sanchez Vicario, dvojka za Grafom na French Open a Wimbledone, začína hrať proti kvalifikantke v štvrtine žrebu, v ktorej je mladý talent Martina Hingisová, 16. nasadená, pred pravdepodobným štvrťfinálovým súbojom so siedmou nasadenou veteránkou Janou. Novotná .","[1, 2, 0, 0

In [219]:

def get_all_phrases(tokens,tags):
    if tags is None:
        return None
    iterator = iter(enumerate(tags))
    prhases=[]
    current_phrase=None
    B_tag=None
    phrases_word_count = 0
    max_i = len(tags)-1
    for i,tag in iterator:
        if (tag==0):
            if B_tag and phrases_word_count>2 :
                prhases.append(current_phrase)
            current_phrase=None
            B_tag=None
            phrases_word_count=0
        elif tag % 2==1: #is beggining
            B_tag=tag    
            current_phrase=tokens[i]
            phrases_word_count=1
        elif B_tag is not None and tag==B_tag+1: #if next tag is I- version of that B-tag
            current_phrase+=f" {tokens[i]}"
            phrases_word_count+=1
  
    if len(prhases)>0:
        return prhases
      
todo = pd.DataFrame(df_annotated)
#fill_gaps_in_anotation([1,0,2,0])
todo["fixed_tags"] =  df_annotated["new_ner_tags"].apply(lambda x: fill_gaps_in_anotation(x, return_none_if_nothing_to_repair=True))
#todo[todo["fixed_tokens"].notnull()]
todo["fixed_phrases"]  = todo[["new_tokens","fixed_tags"]].apply(lambda x: get_all_phrases(x[0],x[1]),axis=1)
todo[todo["fixed_phrases"].notnull()]

,id,tokens,ner_tags,translated,new_ner_tags,new_tokens,fixed_tags,fixed_phrases
40,40,"[Consultations, should, be, held, to, set, the, time, and, format, of, the, talks, ,, the, official, Xinhua, news, agency, quoted, Tang, Shubei, ,, executive, vice, chairman, of, the, Association, for, Relations, Across, the, Taiwan, Straits, ,, as, saying, late, on, Wednesday, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0]","Mali by sa uskutočniť konzultácie, aby sa určil čas a formát rozhovorov, citovala oficiálna tlačová agentúra Xinhua v stredu neskoro večer Tang Shubei, výkonný podpredseda Asociácie pre vzťahy naprieč Taiwanským prielivom.","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2, 0, 0, 0, 3, 4, 4, 0, 4, 4, 0]","[Mali, by, sa, uskutočniť, konzultácie, ,, aby, sa, určil, čas, a, formát, rozhovorov, ,, citovala, oficiálna, tlačová, agentúra, Xinhua, v, stredu, neskoro, večer, Tang, Shubei, ,, výkonný, podpredseda, Asociácie, pre, vzťahy, naprieč, Taiwanským, prielivom, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 2, 0, 0, 0, 3, 4, 4, 4, 4, 4, 0]",[Asociácie pre vzťahy naprieč Taiwanským prielivom]
242,242,"[--, E., Auchard, ,, Wall, Street, bureau, ,, 212-859-1736]","[0, 1, 2, 0, 3, 4, 4, 0, 0]","-- E. Auchard , kancelária Wall Street , 212-859-1736","[0, 0, 1, 0, 2, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0]","[-, -, E, ., Auchard, ,, kancelária, Wall, Street, ,, 212, -, 859, -, 1736]","[0, 0, 1, 2, 2, 0, 0, 5, 6, 0, 0, 0, 0, 0, 0]",[E . Auchard]
1127,1127,"[Sanchez, Vicario, ,, runner-up, to, Graf, at, the, French, Open, and, Wimbledon, ,, begins, play, against, a, qualifier, in, a, quarter, of, the, draw, that, includes, young, talent, Martina, Hingis, ,, the, 16th, seed, ,, before, a, probable, quarter-final, clash, with, seventh-seeded, veteran, Jana, Novotna, .]","[1, 2, 0, 0, 0, 1, 0, 0, 7, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0]","Sanchez Vicario, dvojka za Grafom na French Open a Wimbledone, začína hrať proti kvalifikantke v štvrtine žrebu, v ktorej je mladý talent Martina Hingisová, 16. nasadená, pred pravdepodobným štvrťfinálovým súbojom so siedmou nasadenou veteránkou Janou. Novotná .","[1, 2, 0, 0, 0, 1, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0]","[Sanchez, Vicario, ,, dvojka, za, Grafom, na, French, Open, a, Wimbledone, ,, začína, hrať, proti, kvalifikantke, v, štvrtine, žrebu, ,, v, ktorej, je, mladý, talent, Martina, Hingisová, ,, 16, ., nasadená, ,, pred, pravdepodobným, štvrťfinálovým, súbojom, so, siedmou, nasadenou, veteránkou, Janou, ., Novotná, .]","[1, 2, 0, 0, 0, 1, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0]",[Janou . Novotná]
1259,1259,"[Itar-Tass, news, agency, said, it, provided, for, the, disengagement, of, Russian, and, rebel, forces, in, Chechnya, .]","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 5, 0]","Tlačová agentúra Itar-Tass uviedla, že zabezpečila stiahnutie ruských a povstaleckých síl v Čečensku.","[0, 0, 3, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0]","[Tlačová, agentúra, Itar, -, Tass, uviedla, ,, že, zabezpečila, stiahnutie, ruských, a, povstaleckých, síl, v, Čečensku, .]","[0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0]",[Itar - Tass]
1260,1260,"[Lebed, aide, says, Russian-Chechen, talks, going, well, .]","[1, 0, 0, 7, 0, 0, 0, 0]","Pobočník Lebed hovorí, že rusko-čečenské rozhovory idú dobre.","[0, 1, 0, 0, 0, 7, 0, 8, 0, 0, 0, 0]","[Pobočník, Lebed, hovorí, ,, že, rusko, -, čečenské, rozhovory, idú, dobre, .]","[0, 1, 0, 0, 0, 7, 8, 8, 0, 0, 0, 0]",[rusko - čečenské]
1345,1345,"[One, official, told, me, ', you, have, to, either, destroy, the, books, or, return, them, to, Amman, ', ,, "", Daoud, Makkawi, ,, owner, of, the, Nablus-based, al-Risal

In [224]:

todo.at[4359,"fixed_tags"] = [3, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [225]:
#OK... I commit to this
rowsToUpdate=todo[todo["fixed_tags"].notnull()].index
print(len(rowsToUpdate))
#df_annotated.loc[rowsToUpdate]
df_annotated.at[rowsToUpdate,"new_ner_tags"] = todo.loc[rowsToUpdate]["fixed_tags"]
df_annotated

42


,id,tokens,ner_tags,translated,new_ner_tags,new_tokens,fixed_tags
0,0,"[EU, rejects, German, call, to, boycott, British, lamb, .]","[3, 0, 7, 0, 0, 0, 7, 0, 0]",EÚ odmieta nemeckú výzvu na bojkot britského jahňacieho mäsa.,"[3, 0, 7, 0, 0, 0, 7, 0, 0]","[EÚ, odmieta, nemeckú, výzvu, na, bojkot, britského, jahňacieho, mäsa.]",None
1,1,"[Peter, Blackburn]","[1, 2]",Peter Blackburn,"[1, 2]","[Peter, Blackburn]",None
2,2,"[BRUSSELS, 1996-08-22]","[5, 0]",BRUSEL 1996-08-22,"[5, 0]","[BRUSEL, 1996-08-22]",None
3,3,"[The, European, Commission, said, on, Thursday, it, disagreed, with, German, advice, to, consumers, to, shun, British, lamb, until, scientists, determine, whether, mad, cow, disease, can, be, transmitted, to, sheep, .]","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Európska komisia vo štvrtok uviedla, že nesúhlasí s nemeckou radou spotrebiteľom, aby sa vyhýbali britskej jahňacine, kým vedci nezistia, či sa choroba šialených kráv môže preniesť na ovce.","[3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Európska, komisia, vo, štvrtok, uviedla,, že, nesúhlasí, s, nemeckou, radou, spotrebiteľom,, aby, sa, vyhýbali, britskej, jahňacine,, kým, vedci, nezistia,, či, sa, choroba, šialených, kráv, môže, preniesť, na, ovce.]",None
4,4,"[Germany, 's, representative, to, the, European, Union, 's, veterinary, committee, Werner, Zwingmann, said, on, Wednesday, consumers, should, buy, sheepmeat, from, countries, other, than, Britain, until, the, scientific, advice, was, clearer, .]","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]","Zástupca Nemecka vo veterinárnom výbore Európskej únie Werner Zwingmann v stredu povedal, že spotrebitelia by mali kupovať ovčie mäso z iných krajín ako z Británie, kým nebudú jasné vedecké odporúčania.","[0, 5, 0, 0, 0, 3, 4, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0]","[Zástupca, Nemecka, vo, veterinárnom, výbore, Európskej, únie, Werner, Zwingmann, v, stredu, povedal,, že, spotrebitelia, by, mali, kupovať, ovčie, mäso, z, iných, krajín, ako, z, Británie,, kým, nebudú, jasné, vedecké, odporúčania.]",None
...,...,...,...,...,...,...,...
14036,14036,"[on, Friday, :]","[0, 0, 0]",v piatok :,"[0, 0, 0]","[v, piatok, :]",None
14037,14037,"[Division, two]","[0, 0]",Divízia dva,"[0, 0]","[Divízia, dva]",None
14038,14038,"[Plymouth, 2, Preston, 1]","[3, 0, 3, 0]",Plymouth 2 Preston 1,"[3, 0, 3, 0]","[Plymouth, 2, Preston, 1]",None
14039,14039,"[Division, three]","[0, 0]",Divízia tri,"[0, 0]","[Divízia, tri]",None


In [226]:
df_annotated=df_annotated.drop("fixed_tags",1)

/var/folders/h5/y7fcd0xd4x71d0639gs2z5c00000gp/T/ipykernel_41491/3696890674.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_annotated=df_annotated.drop("fixed_tags",1)


In [293]:
#save
df_annotated.to_pickle(f"df_annotated_{split_name}.picle")
df_annotated= pd.read_pickle(f"df_annotated_{split_name}.picle")
df_annotated

,id,tokens,ner_tags,translated,new_ner_tags,new_tokens
0,0,"[EU, rejects, German, call, to, boycott, British, lamb, .]","[3, 0, 7, 0, 0, 0, 7, 0, 0]",EÚ odmieta nemeckú výzvu na bojkot britského jahňacieho mäsa.,"[3, 0, 7, 0, 0, 0, 7, 0, 0]","[EÚ, odmieta, nemeckú, výzvu, na, bojkot, britského, jahňacieho, mäsa.]"
1,1,"[Peter, Blackburn]","[1, 2]",Peter Blackburn,"[1, 2]","[Peter, Blackburn]"
2,2,"[BRUSSELS, 1996-08-22]","[5, 0]",BRUSEL 1996-08-22,"[5, 0]","[BRUSEL, 1996-08-22]"
3,3,"[The, European, Commission, said, on, Thursday, it, disagreed, with, German, advice, to, consumers, to, shun, British, lamb, until, scientists, determine, whether, mad, cow, disease, can, be, transmitted, to, sheep, .]","[0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","Európska komisia vo štvrtok uviedla, že nesúhlasí s nemeckou radou spotrebiteľom, aby sa vyhýbali britskej jahňacine, kým vedci nezistia, či sa choroba šialených kráv môže preniesť na ovce.","[3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Európska, komisia, vo, štvrtok, uviedla,, že, nesúhlasí, s, nemeckou, radou, spotrebiteľom,, aby, sa, vyhýbali, britskej, jahňacine,, kým, vedci, nezistia,, či, sa, choroba, šialených, kráv, môže, preniesť, na, ovce.]"
4,4,"[Germany, 's, representative, to, the, European, Union, 's, veterinary, committee, Werner, Zwingmann, said, on, Wednesday, consumers, should, buy, sheepmeat, from, countries, other, than, Britain, until, the, scientific, advice, was, clearer, .]","[5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0]","Zástupca Nemecka vo veterinárnom výbore Európskej únie Werner Zwingmann v stredu povedal, že spotrebitelia by mali kupovať ovčie mäso z iných krajín ako z Británie, kým nebudú jasné vedecké odporúčania.","[0, 5, 0, 0, 0, 3, 4, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0]","[Zástupca, Nemecka, vo, veterinárnom, výbore, Európskej, únie, Werner, Zwingmann, v, stredu, povedal,, že, spotrebitelia, by, mali, kupovať, ovčie, mäso, z, iných, krajín, ako, z, Británie,, kým, nebudú, jasné, vedecké, odporúčania.]"
...,...,...,...,...,...,...
14036,14036,"[on, Friday, :]","[0, 0, 0]",v piatok :,"[0, 0, 0]","[v, piatok, :]"
14037,14037,"[Division, two]","[0, 0]",Divízia dva,"[0, 0]","[Divízia, dva]"
14038,14038,"[Plymouth, 2, Preston, 1]","[3, 0, 3, 0]",Plymouth 2 Preston 1,"[3, 0, 3, 0]","[Plymouth, 2, Preston, 1]"
14039,14039,"[Division, three]","[0, 0]",Divízia tri,"[0, 0]","[Divízia, tri]"


# Create Dataset

In [115]:
from datasets import Dataset, DatasetDict, load_dataset,DatasetBuilder

In [294]:
df_dict = {
"train": pd.read_pickle("df_annotated_train.picle"),
"test": pd.read_pickle("df_annotated_test.picle"),
"valid": pd.read_pickle("df_annotated_val.picle"),
}


In [295]:
for key in df_dict:
    df = df_dict[key]
    df["tokens"]=df["new_tokens"]
    df["ner_tags"]=df["new_ner_tags"]
    df_dict[key] = df[["id","tokens","ner_tags"]]
    

In [296]:
df_dict["train"]

,id,tokens,ner_tags
0,0,"[EÚ, odmieta, nemeckú, výzvu, na, bojkot, britského, jahňacieho, mäsa.]","[3, 0, 7, 0, 0, 0, 7, 0, 0]"
1,1,"[Peter, Blackburn]","[1, 2]"
2,2,"[BRUSEL, 1996-08-22]","[5, 0]"
3,3,"[Európska, komisia, vo, štvrtok, uviedla,, že, nesúhlasí, s, nemeckou, radou, spotrebiteľom,, aby, sa, vyhýbali, britskej, jahňacine,, kým, vedci, nezistia,, či, sa, choroba, šialených, kráv, môže, preniesť, na, ovce.]","[3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,4,"[Zástupca, Nemecka, vo, veterinárnom, výbore, Európskej, únie, Werner, Zwingmann, v, stredu, povedal,, že, spotrebitelia, by, mali, kupovať, ovčie, mäso, z, iných, krajín, ako, z, Británie,, kým, nebudú, jasné, vedecké, odporúčania.]","[0, 5, 0, 0, 0, 3, 4, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0]"
...,...,...,...
14036,14036,"[v, piatok, :]","[0, 0, 0]"
14037,14037,"[Divízia, dva]","[0, 0]"
14038,14038,"[Plymouth, 2, Preston, 1]","[3, 0, 3, 0]"
14039,14039,"[Divízia, tri]","[0, 0]"


In [297]:
for key in df_dict:
    df = df_dict[key] 
    df.to_json(f"./output/conll2003-sk-ner/{key}.json", orient="records",lines=True)

In [245]:
for key in df_dict:
    try:
        Dataset.from_json(f"./output/conll03-sk-ner/{key}.json")
    except:
        print(key)

Using custom data configuration default-bae20d7f30cf7c41


100%|██████████| 1/1 [00:00<00:00, 548.56it/s]


Dataset json downloaded and prepared to /Users/juraj/.cache/huggingface/datasets/json/default-bae20d7f30cf7c41/0.0.0. Subsequent calls will reuse this data.


Using custom data configuration default-cb3629589fc68b8c


100%|██████████| 1/1 [00:00<00:00, 422.30it/s]


Dataset json downloaded and prepared to /Users/juraj/.cache/huggingface/datasets/json/default-cb3629589fc68b8c/0.0.0. Subsequent calls will reuse this data.


Using custom data configuration default-8c316ebfd1a4ca32


100%|██████████| 1/1 [00:00<00:00, 565.57it/s]


Dataset json downloaded and prepared to /Users/juraj/.cache/huggingface/datasets/json/default-8c316ebfd1a4ca32/0.0.0. Subsequent calls will reuse this data.


In [116]:
dataset = load_dataset(f"./output/conll2003-sk-ner")
dataset

100%|██████████| 3/3 [00:00<00:00, 304.14it/s]


Dataset conll2003 downloaded and prepared to /Users/juraj/.cache/huggingface/datasets/conll2003/conll2003-SK-NER/1.0.0/84afbe491c93c9db25904122748bbefeb88fce1f1437243e1a6951280d367c35. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 370.84it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3453
    })
})

In [127]:
dataset["train"].features["ner_tags"].feature

ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None)